In [1]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE 
import pandas as pd 
import numpy as np 

In [65]:
from bokeh.io import push_notebook
from bokeh.plotting import figure, ColumnDataSource, output_notebook, output_file, show, save 
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool
from bokeh.palettes import brewer
output_notebook()

Loading BokehJS ...

In [2]:
data = pd.read_csv("../../dataset.csv", error_bad_lines = False)

In [3]:
data

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
0,0,0,4786321,2004-01-01 00:01:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,06,2004.0,41.734106,-87.563621
1,1,1,4676906,2003-03-01 00:00:00,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,26,2003.0,41.817229,-87.637328
2,2,2,4789749,2004-06-20 11:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,20,2004.0,41.915620,-87.694019
3,3,3,4789765,2004-12-30 20:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,06,2004.0,41.919054,-87.752178
4,4,4,4677901,2003-05-01 01:00:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,06,2003.0,41.691785,-87.635116
5,5,5,4838048,2004-08-01 00:01:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,10.0,06,2004.0,41.853447,-87.712625
6,6,6,4791194,2001-01-01 11:00:00,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,02,2001.0,41.687020,-87.608445
7,7,7,4679521,2003-03-15 00:00:00,5007,OTHER OFFENSE,OTHER WEAPONS VIOLATION,RESIDENCE PORCH/HALLWAY,False,False,22.0,26,2003.0,41.729712,-87.653159
8,8,8,4792195,2004-09-16 10:00:00,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,18.0,06,2004.0,41.902862,-87.636098
9,9,9,4680124,2003-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,11.0,06,2003.0,41.869772,-87.708180


In [4]:
data.describe()

,Unnamed: 0,Unnamed: 0.1,ID,District,Year,Latitude,Longitude
count,7.941281e+06,7.941281e+06,7.941281e+06,7.941281e+06,7.941281e+06,7.941281e+06,7.941281e+06
mean,3.970640e+06,3.970640e+06,5.926071e+06,1.131216e+01,2.007672e+03,4.184150e+01,-8.767201e+01
std,2.292451e+06,2.292451e+06,2.568290e+06,6.944530e+00,4.064012e+00,9.135010e-02,6.319202e-02
min,0.000000e+00,0.000000e+00,6.340000e+02,1.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01
25%,1.985320e+06,1.985320e+06,3.853210e+06,6.000000e+00,2.005000e+03,4.176862e+01,-8.771409e+01
50%,3.970640e+06,3.970640e+06,6.165079e+06,1.000000e+01,2.008000e+03,4.185387e+01,-8.766638e+01
75%,5.955960e+06,5.955960e+06,7.716590e+06,1.700000e+01,2.010000e+03,4.190697e+01,-8.762858e+01
max,7.941280e+06,7.941280e+06,1.082788e+07,3.100000e+01,2.017000e+03,4.202291e+01,-8.752453e+01


In [ ]:
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Usage')
plt.title('Programming language usage')
 
plt.show()

In [5]:
data["District"].value_counts()

8.0     550015
11.0    498783
7.0     476533
25.0    463944
6.0     457671
4.0     453894
3.0     407874
9.0     397943
12.0    386276
2.0     378298
5.0     354569
15.0    352589
19.0    351541
18.0    337201
10.0    335495
14.0    314646
1.0     291082
16.0    263446
22.0    263058
24.0    237985
17.0    230957
20.0    137310
31.0       159
13.0         5
21.0         4
23.0         3
Name: District, dtype: int64

In [7]:
data['Date'] = pd.to_datetime(data['Date'])

In [8]:
data['Date'].dt.year

0          2004
1          2003
2          2004
3          2004
4          2003
5          2004
6          2001
7          2003
8          2004
9          2003
10         2003
11         2004
12         2003
13         2004
14         2003
15         2003
16         2003
17         2001
18         2001
19         2002
20         2001
21         2004
22         2003
23         2003
24         2003
25         2001
26         2003
27         2004
28         2004
29         2004
           ... 
7941251    2016
7941252    2016
7941253    2016
7941254    2016
7941255    2016
7941256    2016
7941257    2016
7941258    2016
7941259    2016
7941260    2016
7941261    2016
7941262    2016
7941263    2016
7941264    2016
7941265    2016
7941266    2016
7941267    2016
7941268    2016
7941269    2016
7941270    2016
7941271    2016
7941272    2016
7941273    2016
7941274    2016
7941275    2016
7941276    2016
7941277    2016
7941278    2016
7941279    2016
7941280    2016
Name: Date, dtype: int64

In [30]:
D8 = data.loc[data.District == 8.0]

In [31]:
D8["Date"].dt.year.value_counts()

2008    61556
2009    57152
2006    57059
2010    50177
2007    44244
2001    36632
2002    32147
2003    31724
2005    31500
2004    26133
2011    25366
2012    22661
2013    20222
2014    18194
2016    17284
2015    17217
2017      747
Name: Date, dtype: int64

In [6]:
from sklearn.cluster import KMeans

In [23]:
df2 = originalDf.drop('FBI Code', 1)

In [24]:
df2

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Latitude,Longitude
0,0,0,4786321,2004-01-01 00:01:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,41.734106,-87.563621
1,1,1,4676906,2003-03-01 00:00:00,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,2003.0,41.817229,-87.637328
2,2,2,4789749,2004-06-20 11:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,2004.0,41.915620,-87.694019
3,3,3,4789765,2004-12-30 20:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,2004.0,41.919054,-87.752178
4,4,4,4677901,2003-05-01 01:00:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,2003.0,41.691785,-87.635116
5,5,5,4838048,2004-08-01 00:01:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,APARTMENT,False,False,10.0,2004.0,41.853447,-87.712625
6,6,6,4791194,2001-01-01 11:00:00,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,2001.0,41.687020,-87.608445
7,7,7,4679521,2003-03-15 00:00:00,5007,OTHER OFFENSE,OTHER WEAPONS VIOLATION,RESIDENCE PORCH/HALLWAY,False,False,22.0,2003.0,41.729712,-87.653159
8,8,8,4792195,2004-09-16 10:00:00,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,18.0,2004.0,41.902862,-87.636098
9,9,9,4680124,2003-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,11.0,2003.0,41.869772,-87.708180


In [71]:
expandedData_sub = expandedData.sample(5000)

In [79]:
df = expandedData_sub
df

,Unnamed: 0,ID,Arrest,Domestic,District,Year,Latitude,Longitude,Months,Hours,...,OBSCENITY,KIDNAPPING,INTIMIDATION,RITUALISM,DOMESTIC VIOLENCE,OTHER NARCOTIC VIOLATION,PUBLIC INDECENCY,NON-CRIMINAL,HUMAN TRAFFICKING,CONCEALED CARRY LICENSE VIOLATION
5757862,5757862,7296701,0,0,16.0,2009.0,41.951825,-87.807236,12,17,...,0,0,0,0,0,0,0,0,0,0
60327,60327,1386033,0,0,14.0,2001.0,41.920972,-87.707240,2,18,...,0,0,0,0,0,0,0,0,0,0
3385128,3385128,3979627,0,0,22.0,2005.0,41.725952,-87.662789,5,10,...,0,0,0,0,0,0,0,0,0,0
1171327,1171327,2779752,0,0,25.0,2003.0,41.908200,-87.773142,6,11,...,0,0,0,0,0,0,0,0,0,0
4773078,4773078,7552656,0,0,2.0,2010.0,41.797277,-87.599693,4,12,...,0,0,0,0,0,0,0,0,0,0
1988492,1988492,4769850,0,0,14.0,2006.0,41.927351,-87.704598,5,2,...,0,0,0,0,0,0,0,0,0,0
1837023,1837023,3626721,0,0,9.0,2004.0,41.837453,-87.635304,10,17,...,0,0,0,0,0,0,0,0,0,0
2344236,2344236,5387343,1,0,18.0,2007.0,41.922510,-87.663226,3,22,...,0,0,0,0,0,0,0,0,0,0
5326763,5326763,6612215,1,0,14.0,2008.0,41.910136,-87.705005,11,21,...,0,0,0,0,0,0,0,0,0,0
2965549,2965549,5559754,0,0,12.0,2007.0,41.884119,-87.702183,5,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
pd.unique(data['District'])

array([  4.,   9.,  14.,  25.,  22.,  10.,   5.,  18.,  11.,  20.,   8.,
         7.,   1.,  16.,  15.,   3.,   6.,   2.,  19.,  12.,  24.,  17.,
        31.,  21.,  13.,  23.])

In [13]:
subByYear2001 = data[data.Date.dt.year == 2001]
subByYear2001

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
6,6,6,4791194,2001-01-01 11:00:00,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,True,5.0,02,2001.0,41.687020,-87.608445
17,17,17,4803028,2001-01-01 00:00:00,1753,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,True,False,8.0,02,2001.0,41.791519,-87.729099
18,18,18,4803606,2001-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,9.0,06,2001.0,41.834523,-87.682642
20,20,20,4809221,2001-06-16 08:00:00,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,True,False,7.0,02,2001.0,41.778540,-87.674036
25,25,25,4811727,2001-06-01 08:00:00,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,15.0,17,2001.0,41.896809,-87.762472
32,32,32,4821866,2001-07-25 10:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,True,14.0,06,2001.0,41.921795,-87.713269
33,33,33,4822001,2001-06-23 20:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,7.0,06,2001.0,41.772187,-87.638570
50,50,50,4835774,2001-01-01 00:00:00,0842,THEFT,AGG: FINANCIAL ID THEFT,OTHER,False,False,8.0,06,2001.0,41.748448,-87.679203
63,63,63,4850450,2001-05-01 00:01:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,4.0,20,2001.0,41.657828,-87.544575
118,118,118,4886733,2001-10-01 00:01:00,0842,THEFT,AGG: FINANCIAL ID THEFT,RESIDENCE,False,False,7.0,06,2001.0,41.774876,-87.636719


In [46]:
sub2001Df = subByYear2001.sample(10000)

In [57]:
year2002 = data[data.Date.dt.year == 2002]

In [11]:
from kmodes.kmodes import KModes

In [43]:
km = KModes(n_clusters=4, init = "Huang", n_init = 5, verbose=1)

In [48]:
clusters = km.fit_predict(sub2001Df)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1493, cost: 99602.0
Run 1, iteration: 2/100, moves: 179, cost: 99600.0
Run 1, iteration: 3/100, moves: 0, cost: 99600.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1893, cost: 99919.0
Run 2, iteration: 2/100, moves: 13, cost: 99919.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 65, cost: 99359.0
Run 3, iteration: 2/100, moves: 0, cost: 99359.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 872, cost: 99169.0
Run 4, iteration: 2/100, moves: 846, cost: 99165.0
Run 4, iteration: 3/100, moves: 2, cost: 99165.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1120, cost: 99333.0
Run 5, iteration: 2/100, moves: 4, cost: 993

In [49]:
print(km.cluster_centroids_)

[[2178 2178 1312670 Timestamp('2001-04-01 00:01:00') '1811' 'NARCOTICS'
  'POSS: CANNABIS 30GMS OR LESS' 'STREET' True False 25.0 '18' 2001.0
  41.976200173 -87.905312411]
 [618 618 1313031 Timestamp('2001-01-01 00:00:00') '0820' 'THEFT'
  '$500 AND UNDER' 'STREET' False False 8.0 '06' 2001.0 41.754658085
  -87.741385006]
 [8204 8204 1310610 Timestamp('2001-07-16 22:00:00') '0810' 'THEFT'
  'OVER $500' 'STREET' False False 19.0 '06' 2001.0 41.789387343
  -87.741410303]
 [590 590 1319011 Timestamp('2001-02-21 09:00:00') '0460' 'BATTERY'
  'SIMPLE' 'RESIDENCE' False False 2.0 '08B' 2001.0 41.804149939
  -87.62609605]]


In [32]:
D8

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
12,12,12,4684808,2003-01-01 00:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,8.0,06,2003.0,41.783164,-87.727773
15,15,15,4687321,2003-11-15 09:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,APARTMENT,False,False,8.0,06,2003.0,41.778921,-87.706365
17,17,17,4803028,2001-01-01 00:00:00,1753,OFFENSE INVOLVING CHILDREN,SEX ASSLT OF CHILD BY FAM MBR,RESIDENCE,True,False,8.0,02,2001.0,41.791519,-87.729099
23,23,23,4696148,2003-01-01 09:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,8.0,06,2003.0,41.745268,-87.699825
45,45,45,4833086,2004-06-29 00:01:00,1120,DECEPTIVE PRACTICE,FORGERY,BANK,False,True,8.0,10,2004.0,41.778283,-87.715424
50,50,50,4835774,2001-01-01 00:00:00,0842,THEFT,AGG: FINANCIAL ID THEFT,OTHER,False,False,8.0,06,2001.0,41.748448,-87.679203
56,56,56,4842389,2004-10-13 12:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,True,8.0,06,2004.0,41.778283,-87.715424
61,61,61,4849736,2004-07-09 00:01:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,SIDEWALK,False,False,8.0,06,2004.0,41.778283,-87.715424
97,97,97,4752685,2003-11-01 12:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,STREET,False,False,8.0,06,2003.0,41.765924,-87.689395
115,115,115,4762257,2003-07-01 01:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,True,True,8.0,20,2003.0,41.776925,-87.716371


In [36]:
D8_sub = D8.sample(10000)

In [37]:
D8_sub

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
2984985,2984985,2984985,5592428,2007-06-06 13:00:00,0870,THEFT,POCKET-PICKING,SMALL RETAIL STORE,False,False,8.0,06,2007.0,41.745071,-87.741217
6272523,6272523,6272523,8104568,2011-06-09 17:00:00,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,8.0,14,2011.0,41.788635,-87.726093
1298586,1298586,1298586,2941949,2003-08-18 16:45:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,8.0,06,2003.0,41.763733,-87.689336
2640250,2640250,2640250,4960478,2006-08-30 21:00:00,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE-GARAGE,False,False,8.0,05,2006.0,41.745507,-87.705935
6557421,6557421,6557421,9994826,2015-03-15 11:00:00,0560,ASSAULT,SIMPLE,RESIDENCE,False,True,8.0,08A,2015.0,41.810353,-87.744453
3539953,3539953,3539953,4270119,2005-08-30 18:05:00,0860,THEFT,RETAIL THEFT,SMALL RETAIL STORE,False,False,8.0,06,2005.0,41.789675,-87.801435
4651239,4651239,4651239,7362869,2010-02-14 21:30:00,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,PARKING LOT/GARAGE(NON.RESID.),True,False,8.0,18,2010.0,41.772685,-87.693228
3155017,3155017,3155017,5851740,2007-10-18 11:00:00,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,8.0,26,2007.0,41.776597,-87.701856
2922851,2922851,2922851,5479543,2007-04-12 23:30:00,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,8.0,14,2007.0,41.761738,-87.694173
3171854,3171854,3171854,5878907,2007-11-03 03:00:00,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,8.0,14,2007.0,41.771253,-87.732959


In [44]:
clusters = km.fit_predict(D8_sub)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 2879, cost: 101158.0
Run 1, iteration: 2/100, moves: 855, cost: 101066.0
Run 1, iteration: 3/100, moves: 37, cost: 101066.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 975, cost: 100655.0
Run 2, iteration: 2/100, moves: 121, cost: 100655.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 3401, cost: 99950.0
Run 3, iteration: 2/100, moves: 0, cost: 99950.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 2348, cost: 99630.0
Run 4, iteration: 2/100, moves: 572, cost: 99630.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1772, cost: 100601.0
Run 5, iteration: 2/100, moves: 171, cost: 100601.0
Best run was number 4


In [45]:
print(km.cluster_centroids_)

[[56 56 1375378 Timestamp('2001-02-08 13:00:00') '0820' 'THEFT'
  '$500 AND UNDER' 'STREET' False False 8.0 '06' 2008.0 41.754592961
  -87.74152853700001]
 [611 611 1362084 Timestamp('2001-02-01 20:40:00') '1310' 'CRIMINAL DAMAGE'
  'TO PROPERTY' 'RESIDENCE' False False 8.0 '14' 2006.0 41.77828299100242
  -87.71542426828199]
 [21435 21435 4689558 Timestamp('2006-04-15 19:00:00') '0486' 'BATTERY'
  'DOMESTIC BATTERY SIMPLE' 'RESIDENCE' False True 8.0 '08B' 2008.0
  41.754592961 -87.74152853700001]
 [55805 55805 1403766 Timestamp('2001-02-27 18:00:00') '1811' 'NARCOTICS'
  'POSS: CANNABIS 30GMS OR LESS' 'STREET' True False 8.0 '18' 2010.0
  41.778877155 -87.70884010600003]]


In [51]:
expandedData = pd.read_csv("../../expandedPrimaryType.csv", error_bad_lines = False)

In [54]:
expandedData.head()

,Unnamed: 0,ID,Arrest,Domestic,District,Year,Latitude,Longitude,Months,Hours,...,OBSCENITY,KIDNAPPING,INTIMIDATION,RITUALISM,DOMESTIC VIOLENCE,OTHER NARCOTIC VIOLATION,PUBLIC INDECENCY,NON-CRIMINAL,HUMAN TRAFFICKING,CONCEALED CARRY LICENSE VIOLATION
0,0,4786321,0,0,4.0,2004.0,41.734106,-87.563621,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4676906,0,1,9.0,2003.0,41.817229,-87.637328,3,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4789749,0,0,14.0,2004.0,41.915620,-87.694019,6,11,...,0,0,0,0,0,0,0,0,0,0
3,3,4789765,0,0,25.0,2004.0,41.919054,-87.752178,12,20,...,0,0,0,0,0,0,0,0,0,0
4,4,4677901,0,0,22.0,2003.0,41.691785,-87.635116,5,1,...,0,0,0,0,0,0,0,0,0,0


In [56]:
expandedData.shape

(7941281, 46)

In [58]:
sub_year2002 = year2002.sample(10000)

In [59]:
clusters = km.fit_predict(sub_year2002)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 1298, cost: 100974.0
Run 1, iteration: 2/100, moves: 81, cost: 100974.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 1381, cost: 104037.0
Run 2, iteration: 2/100, moves: 20, cost: 104037.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 1052, cost: 103937.0
Run 3, iteration: 2/100, moves: 0, cost: 103937.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 1274, cost: 102163.0
Run 4, iteration: 2/100, moves: 2, cost: 102162.0
Run 4, iteration: 3/100, moves: 0, cost: 102162.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 2357, cost: 98721.0
Run 5, iteration: 2/100, moves: 503, cost: 98721.0
Best run was number 5


In [60]:
print(km.cluster_centroids_)

[[905 905 8017383 Timestamp('2002-01-01 00:01:00') '0820' 'THEFT'
  '$500 AND UNDER' 'STREET' False False 8.0 '06' 2002.0 41.68769366978374
  -87.62293524998104]
 [3496 3496 1677 Timestamp('2002-01-06 09:00:00') '1310' 'CRIMINAL DAMAGE'
  'TO PROPERTY' 'STREET' False False 16.0 '14' 2002.0 41.976290414
  -87.905227221]
 [2170 2170 1327 Timestamp('2002-01-02 18:00:00') '1811' 'NARCOTICS'
  'POSS: CANNABIS 30GMS OR LESS' 'STREET' True False 11.0 '18' 2002.0
  41.68769366978374 -87.62293524998104]
 [3406 3406 1922858 Timestamp('2002-09-30 14:00:00') '0460' 'BATTERY'
  'SIMPLE' 'RESIDENCE' False False 7.0 '08B' 2002.0 41.68769366978374
  -87.62293524998104]]


In [61]:
expandedData_sub = expandedData.sample(10000)

In [64]:
expandedData_sub

,Unnamed: 0,ID,Arrest,Domestic,District,Year,Latitude,Longitude,Months,Hours,...,OBSCENITY,KIDNAPPING,INTIMIDATION,RITUALISM,DOMESTIC VIOLENCE,OTHER NARCOTIC VIOLATION,PUBLIC INDECENCY,NON-CRIMINAL,HUMAN TRAFFICKING,CONCEALED CARRY LICENSE VIOLATION
3444625,3444625,4092017,0,0,19.0,2005.0,41.938029,-87.658172,6,21,...,0,0,0,0,0,0,0,0,0,0
6035884,6035884,7729451,1,0,8.0,2010.0,41.742523,-87.700669,9,0,...,0,0,0,0,0,0,0,0,0,0
6917375,6917375,8993361,0,0,1.0,2013.0,41.876488,-87.624278,1,19,...,0,0,0,0,0,0,0,0,0,0
7097876,7097876,9284400,0,0,16.0,2013.0,41.938334,-87.784908,8,9,...,0,0,0,0,0,0,0,0,0,0
4374938,4374938,6935949,0,0,4.0,2009.0,41.758380,-87.579802,5,7,...,0,0,0,0,0,0,0,0,0,0
5385499,5385499,6706597,0,0,25.0,2009.0,41.931589,-87.742446,1,12,...,0,0,0,0,0,0,0,0,0,0
6207456,6207456,8004405,1,0,6.0,2011.0,41.751188,-87.610740,4,20,...,0,0,0,0,0,0,0,0,0,0
5753364,5753364,7289376,0,0,3.0,2009.0,41.779277,-87.615601,12,12,...,0,0,0,0,0,0,0,0,0,0
3245749,3245749,7767620,0,0,8.0,2005.0,41.778283,-87.715424,10,20,...,0,0,0,0,0,0,0,0,0,0
3583651,3583651,4356511,0,0,12.0,2005.0,41.899663,-87.670143,10,7,...,0,0,0,0,0,0,0,0,0,0


In [74]:
'''Function that fitTransforms a TSNE 
and returns the fitTransformed modeled.
User can specify learning rate and dataframe 
'''
def fitTSNE(learningRate, df): 
    model = TSNE(learning_rate = learningRate, perplexity = 40)
    transformed = model.fit_transform(df)
    return transformed

In [75]:
'''Function that plots a TSNE
and demonstrates the modeled plot
User can specified the transformed df
and the list of labels
'''
def plotTSNE(transformed, colorLabels):
    xs = transformed[:,0]
    ys = transformed[:, 1]
    plt.scatter(xs, ys, c = colorLabels)
    plt.show()

In [76]:
def bokehTSNE(transformed, data, graphTitle, categoryName):
    tsne_df = pd.DataFrame(transformed, columns=['Component 1', 'Component2'], index =data.index)
    data_all = pd.concat([data, tsne_df], axis=1)
    category = categoryName
    
    #set color by given category nam e
    category_items = data_all[category].unique()
    palette = brewer['Set3'][len(category_items) + 1]
    colormap = dict(zip(category_items, palette))
    data_all['color'] = data_all[category].map(colormap)
    title = graphTitle

    #the transformed data
    source = ColumnDataSource(data_all)


    hover = HoverTool(tooltips=[(column, '@' + column) for column in reversed(data.columns)])

    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), TapTool(), SaveTool()]

    p = figure(
        tools=tools,
        title=title,
        plot_width=800,
        plot_height=800,
        toolbar_location='below',
        toolbar_sticky=False, )

    p.circle(
        x= 'Component 1',
        y= 'Component2',
        source=source,
        size=10,
        line_color='#333333',
        line_width=0.5,
        fill_alpha=0.8,
        color='color',
        legend=category)

    show(p)

In [78]:
for x in range(50,500, 50): 
    transformed =fitTSNE(x, df)
    bokehTSNE(transformed, df, 'Plot #' + str(x), 'District')

KeyError: 23